In [1]:
import os
import pandas as pd
import seaborn as sns
import plotly.subplots as subplots
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
from sklearn.metrics import r2_score

# XGBoost import
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, confusion_matrix
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

import xgboost as xgb



# joblib import
import joblib


import pickle

# grid_cv import
from sklearn.model_selection import GridSearchCV

# 최대 재귀함수 깊이 설정
import sys
sys.setrecursionlimit(10**7)

# timedelta import
from datetime import timedelta

# datetime import
from datetime import datetime

# go
import plotly.graph_objects as go

# import rc
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm



In [2]:
df= pd.read_csv('data/price/price_baechoo.csv')

# 평균 컬럼 가격 컬럼으로 변경
df.rename(columns={'평균':'가격'}, inplace=True)
# 구분 컬럼 날짜 컬럼으로 변경
df.rename(columns={'구분':'날짜'}, inplace=True)

# df 날짜 인덱스로 설정
df.set_index('날짜', inplace=True)
df

,가격
날짜,
2000-01-03,5600.0
2000-01-04,5600.0
2000-01-05,5620.0
2000-01-06,5700.0
2000-01-07,5670.0
...,...
2022-12-14,5430.5
2022-12-15,5430.5
2022-12-16,5304.0


In [3]:
# train, test 데이터 분리 (train: 2000부터 2022-10, test: 2022-11)
df_train = df.loc['2000':'2022-11']
df_test = df.loc['2022-11':'2022-12']

In [4]:
df_test

,가격
날짜,
2022-11-01,7350.0
2022-11-02,6835.5
2022-11-03,7142.0
2022-11-04,7381.5
2022-11-07,7330.0
2022-11-08,7193.5
2022-11-09,6760.0
2022-11-10,6952.0
2022-11-11,6992.0


In [5]:
df_train

,가격
날짜,
2000-01-03,5600.0
2000-01-04,5600.0
2000-01-05,5620.0
2000-01-06,5700.0
2000-01-07,5670.0
...,...
2022-10-25,8172.0
2022-10-26,8072.0
2022-10-27,8252.0


비 연속적으로 N개 층이 밀리는 방식

In [6]:
def create_candles(df, group_sizes):
    candles = {}
    for group_size in group_sizes:
        candle_df = pd.DataFrame(columns=['시가', '고가', '저가', '종가', '일자'])
        for i in range(0, len(df), group_size):
            temp = df.iloc[i:i+group_size]
            open_price = temp.iloc[0]['가격']
            high_price = temp['가격'].max()
            low_price = temp['가격'].min()
            close_price = temp.iloc[-1]['가격']
            date = temp.iloc[-1].name
            candle_df = candle_df.append({'시가': open_price, '고가': high_price, '저가': low_price, '종가': close_price, '일자': date}, ignore_index=True)
        candles[group_size] = candle_df
    return candles

candles = create_candles(df, [5, 10, 20, 60, 120])
candle_df_5, candle_df_10, candle_df_20, candle_df_60, candle_df_120 = (candles[size] for size in [5, 10, 20, 60, 120])

# Set the index of each dataframe to the '일자' column and add a new '종가_shift' column
for df in candles.values():
    df.set_index('일자', inplace=True)
    df['종가_shift'] = df['종가'].shift(-1)

# Save the last row of each dataframe to a new dataframe
candle_df_lasts = {key: df.iloc[-1] for key, df in candles.items()}

# Transpose and remove missing values from each dataframe
for df in candle_df_lasts.values():
    df = df.T.dropna()

for df in candles.values():
    df.dropna(inplace=True)


In [7]:
candle_df_60

,시가,고가,저가,종가,종가_shift
일자,,,,,
2000-03-15,5600.0,8690.0,5600.0,7990.0,2330.0
2000-05-30,7950.0,8580.0,2330.0,2330.0,4390.0
2000-08-11,2250.0,4870.0,1760.0,4390.0,3695.0
2000-10-27,4390.0,6520.0,3420.0,3695.0,1980.0
2001-01-08,3720.0,3720.0,1750.0,1980.0,3180.0
...,...,...,...,...,...
2021-12-08,13490.0,14570.0,5090.0,8570.0,8644.0
2022-03-08,7890.0,9362.0,7424.0,8644.0,10260.0
2022-06-03,9016.0,11180.0,8504.0,10260.0,23800.0


In [8]:
candle_df_lasts[60]

시가          5592.0
고가          5982.0
저가          5242.0
종가          5764.0
종가_shift       NaN
Name: 2022-12-20, dtype: float64

In [9]:
# 캔들 스틱 차트 그리기 제목추가=candle_df 변수명
def candle_stick_chart(candle_df):
    fig = go.Figure(data=[go.Candlestick(x=candle_df.index,
                                         open=candle_df['시가'],
                                         high=candle_df['고가'],
                                         low=candle_df['저가'],
                                         close=candle_df['종가'])])
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.show()

    

# 캔들 스틱 차트 그리기
candle_stick_chart(candle_df_5)
candle_stick_chart(candle_df_10)
candle_stick_chart(candle_df_20)
candle_stick_chart(candle_df_60)
candle_stick_chart(candle_df_120)


In [10]:
def scale_data(days):
    # select the data for the given number of days
    feature = eval(f'candle_df_{days}')[['시가', '고가', '저가','종가']]
    target = eval(f'candle_df_{days}')[['종가_shift']]
    
    # scale the data
    scaler1 = MinMaxScaler()
    scaler2 = MinMaxScaler()
    feature_scaled = scaler1.fit_transform(feature)
    target_scaled = scaler2.fit_transform(target)
    
    # convert the scaled data to a Pandas DataFrame
    feature_scaled = pd.DataFrame(feature_scaled, columns=feature.columns, index=feature.index)
    target_scaled = pd.DataFrame(target_scaled, columns=target.columns, index=target.index)

    # Save the scaler for the feature data
    joblib.dump(scaler1, f'model/price_candle_scaler1_{days}.pkl')
    joblib.dump(scaler2, f'model/price_candle_scaler2_{days}.pkl')
    
    return feature_scaled, target_scaled

feature_5, target_5 = scale_data(5)
feature_10, target_10 = scale_data(10)
feature_20, target_20 = scale_data(20)
feature_60, target_60 = scale_data(60)
feature_120, target_120 = scale_data(120)

In [11]:
def split_data(feature, target, test_size=0.2, shuffle=False, random_state=42):
  train_X, test_X, train_y, test_y = train_test_split(feature, target, test_size=test_size, shuffle=shuffle, random_state=random_state)
  return train_X, test_X, train_y, test_y

for n in tqdm([5, 10, 20, 60, 120]):
  # Split the data into training and testing sets
  train_X, test_X, train_y, test_y = split_data(eval(f"feature_{n}"), eval(f"target_{n}"))

  # Set the hyperparameter grid for XGBoost
  param_grid = {'n_estimators': [500, 1000, 2000, 5000],
                'learning_rate': [0.01, 0.05, 0.1, 0.2],
                'max_depth': [3, 5, 7, 9, 11],
                'colsample_bytree': [0.5, 0.75, 1.0],
                'subsample': [0.5, 0.75, 1.0],
                'reg_alpha': [0.01, 0.1, 1.0],
                'reg_lambda': [0.01, 0.1, 1.0],
                'booster': ['gbtree', 'gblinear', 'dart']}

  # Create an XGBoost model with GPU acceleration
  xgb = XGBRegressor(tree_method='gpu_hist')

  # Set up the grid search with 5-fold cross-validation
  grid_cv = GridSearchCV(xgb, param_grid, cv=5)

  # Fit the grid search to the data
  grid_cv.fit(train_X, train_y)

  # using the best parameters from the grid search to create a new XGBoost model
  xgb = XGBRegressor(**grid_cv.best_params_, tree_method='gpu_hist')
  xgb.fit(train_X, train_y)

  # Evaluate the model on the test data
  y_pred = xgb.predict(test_X)
  print(f'n = {n}: XGBoost model RMSE = {np.sqrt(mean_squared_error(test_y, y_pred))}')
  print(f'n = {n}: XGBoost model r2_score = {r2_score(test_y, y_pred)}')
  print(f'n = {n}: XGBoost model precision_score = {precision_score(test_y, y_pred)}')
  print(f'n = {n}: XGBoost model recall_score = {recall_score(test_y, y_pred)}')
  print(f'n = {n}: XGBoost model f1_score = {f1_score(test_y, y_pred)}')
  print(f'n = {n}: XGBoost model accuracy_score = {accuracy_score(test_y, y_pred)}')

  # Save the model 
  joblib.dump(xgb, f'model/price_candle_xgb_{n}.pkl')

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
# 시각화
plt.figure(figsize=(12, 9))
plt.plot(test_y, label='actual')
plt.plot(y_pred, label='prediction')
plt.legend()
plt.show()




Make new data

In [ ]:
candle_df_last

In [ ]:
model = pickle.load(open('model/price_candle_XGBoost_static_5days.pkl', 'rb'))
scaler1 = pickle.load(open('model/price_candle_XGBoost_static_5days_scaler1.pkl', 'rb'))
scaler2 = pickle.load(open('model/price_candle_XGBoost_static_5days_scaler2.pkl', 'rb'))

# model predict using test_data
test_data_sc = scaler1.transform(candle_df_last_5)
test_data_sc = pd.DataFrame(test_data_sc, columns=candle_df_last_5.columns, index=candle_df_last_5.index)
y_pred = model.predict(test_data_sc)
# reshape
y_pred = y_pred.reshape(-1, 1)
# inverse transform
y_pred = scaler2.inverse_transform(y_pred)
y_pred

# 10월 31일 
# 2022-11-04	7381.5